# Extract motivational messages 

I conceived of this project the same week as ChatGPT3 exploded into conversations at work, online, and at home. While my original plan had been to practice my NLP skills, I switched my goal to learn how to use this new tool.

Note: Section links work in JupyterLab but do not work when this notebook is viewed in GitHub.

1.  [Load](#Load-transcripts) transcripts from the output of the Notebook 02
2. [Interpret](#Interpret-response-from-ChatGPT3) response from ChatGPT3
3. [Play](#Play-with-the-prompt) with the prompt to get satisfactory response
4. [Summarize](#Summarize-motivational-talks) motivational talks for each video
5. [Save](#Save-the-motivational-messages) the motivational messages
6. [Consider](#Next-steps) next steps



### Load transcripts

In [1]:
import pandas as pd

* I do not push data to code repositories, and encourage you to do the same. To keep everything organized on my machine, I have a `data` folder in my project and include this folder in the project's `.gitignore` file.

In [2]:
filename = '../data/2023-01-effort-motivations.csv'

In [12]:
data = pd.read_csv(filename, parse_dates=['date'])
data.head()

,date,vid,motivation
0,2023-01-02,qLKflJjWDi4,big exhale shift your hips up and forward and...
1,2023-01-03,NzIylrLhkJ4,amazing job rest back on your glutes you did ...
2,2023-01-04,A5kHPgAi3I0,break deep breath nice wide stance hinge forw...
3,2023-01-06,A67VMX9hlmc,give me two minutes here nice wide stance exh...
4,2023-01-07,lpv4RdNefyU,amazing work give me two minutes do not leave...


[Return to top](#Extract-motivational-messages)

### Interpret response from ChatGPT3

* ChatGPT3 has multiple language models available for use. It looks like [Davinci](https://platform.openai.com/docs/models/davinci) will be the right one for this task of interpreting and summarizing.

* Different models will give different results. The creators of an endless, AI-generated, Seinfeld parody [learned this the hard way](https://wapo.st/3I7BSHW) when they switched from Davinci to Curie.

* For this project, I am not worried about generating hateful or harming content. I am taking note of [OpenAI's moderation tools](https://platform.openai.com/docs/guides/moderation/overview) for future work.

* The next three cells are set up.

In [13]:
with open('../config/chat_gpt3_api') as file:
    CHAT_KEY = file.readline()

* Even more important than not pushing data to a code repository is to **never push a key to a code repository**! I have stored the keys in a folder called `config` and added this folder to the project's `.gitignore` file. There are other ways to accomplish this, choose your preferred method.

In [14]:
import openai
openai.api_key = CHAT_KEY
models = openai.Model.list()

In [18]:
test_text = data.loc[0,'motivation']
test_text[:100]

' big exhale shift your hips up and forward and before you head out if this is your first workout or '

* The template for the function in the next cell comes from OpenAI. If you used the playground when you were setting up API access, you already encountered the `temperature` and `max_tokens` parameters.

* I interpret `temperature` as a creativity setting. Set to 0, multiple calls with the same text will return similar (if not identical) responses. Set to 1, the model gets very creative! 

* You can find no better explanation of OpenAI's tokens than the [documentation](https://help.openai.com/en/articles/4936856-what-are-tokens-and-how-to-count-them) from the source. If you want a more general discussion of tokens and tokenization, send "nlp tokens" through your favorite search engine to learn more.

In [26]:
def get_response(text):
    prompt = 'extract the motivational message from the following text and summarize in three sentences: '
    response = openai.Completion.create(
        model='text-davinci-003',
        prompt=prompt+text,
        temperature=0.7,
        max_tokens=25
    )
    return response

In [28]:
response = get_response(test_text)
response

<OpenAIObject text_completion id=cmpl-6hfQV5TTlYKCopTdgWTXgb5zauFPo at 0x7fbf50bd6160> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "\n\nThis motivational message encourages applying effort not just when we're in the mood, but when we show up consistently. It"
    }
  ],
  "created": 1675865367,
  "id": "cmpl-6hfQV5TTlYKCopTdgWTXgb5zauFPo",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 25,
    "prompt_tokens": 464,
    "total_tokens": 489
  }
}

* For a first pass, this is a pretty good summary! Let me see how to extract it from the response.

In [32]:
response['choices'][0]['text']

"\n\nThis motivational message encourages applying effort not just when we're in the mood, but when we show up consistently. It"

[Return to top](#Extract-motivational-messages)

### Play with the prompt

* I notice that it cut off the summary -- no doubt because I limited the `max_tokens` to 25.

* I also notice that it really sounds like a summary! I would rather it sound like a motivational message. I need to adjust the prompt.

In [33]:
def get_response(text):
    prompt = 'extract the motivational message from the following text in three sentences: '
    response = openai.Completion.create(
        model='text-davinci-003',
        prompt=prompt+text,
        temperature=0.7,
        max_tokens=100
    )
    return response

In [34]:
response = get_response(test_text)
response['choices'][0]['text']

'\n\nEffort is an important part of achieving success and making progress towards your goals. Showing up and not giving up is the key to making progress and reaching your goals. Subscribe to the channel to help more people have access to high quality fitness and stay dedicated to putting in the effort every day.'

* This sounds better! However, the model is interpreting some of the boilerplate text as motivational. Adjust the prompt further.

In [42]:
def get_response(text):
    directions = 'extract the motivational message from the following text in three sentences: '
    conditions = 'Ignore sentences that direct viewers to subscribe, like, buy, or comment.'
    response = openai.Completion.create(
        model='text-davinci-003',
        prompt=directions+text+conditions,
        temperature=0.7,
        max_tokens=100
    )
    return response

In [36]:
response = get_response(test_text)
response['choices'][0]['text']

" Motivation comes from within, so remember to stay determined and dedicated to your goals. Show up and never give up, even when you don't feel motivated. Effort will show you that you can do more than you thought and help you reach your goals."

* Not bad! This sounds almost like Sydney herself. I am going to move on with the `get_response` function as defined above.

* Before I get the motivations from all of the videos, I'll write a couple of functions streamline this process.

In [37]:
def extract_text(response):
    '''
    Return the text from the response object
    '''
    text = response['choices'][0]['text']
    return text

* Also, I will be polite by pausing between requests. 

In [38]:
import time

In [39]:
def get_motivation(text, sleep_time=3):
    '''
    Return the extracted motivation from the text, pausing for sleep_time seconds after request.
    '''
    response = get_response(text)
    motivation = extract_text(response)
    time.sleep(sleep_time)
    return motivation

* Test this out...

In [40]:
get_motivation(test_text)

" Motivate yourself to stay dedicated, consistent, and determined with your lifestyle or goal. Put in the effort and don't give up - even when you're not feeling motivated or in the mood. Show up and keep growing together."

[Return to top](#Extract-motivational-messages)

### Summarize motivational talks

* Alright! I am ready to get the motivations for the month! 

* This next call will take a minute because of (1) the sleep parameter and (2) however busy the ChatGPT3 servers are. In fact...the first time I ran the next cell, it returned a rate limit error. That's what happens when the continental US is awake and pinging the servers at 11 am Eastern. I had more success at 4 pm.

In [44]:
motivations = data['motivation'].apply(get_motivation)
motivations[0]

' Motivate yourself to stay dedicated and consistent with your goals by showing up and not giving up. Effort is a vigorous and determined attempt to achieve your goals and can teach you that you can do more than you thought. With consistent effort, you will be able to improve yourself and reach success.'

* Motivation verging on platitudes. It's ok! 

In [45]:
motivations

0      Motivate yourself to stay dedicated and consi...
1      \n\nYou did an amazing job today! Don't forge...
2      Motivational message: Taking the effort to pr...
3     \n\nMotivation is a lifelong journey, and it i...
4     \n\nNever give up, even in times of difficulty...
5     \n\nMotivation comes from pushing yourself to ...
6      Motivational message: Dedicate your time to y...
7     \n\nMotivation comes from within and is based ...
8      Give yourself permission to be proud of yours...
9     \n\nNo matter what is tough, you did incredibl...
10    \n\nConsistency is always your first priority ...
11     \n\nTake small steps towards your goals and d...
12     Motivational message: Staying consistent and ...
13    \n\nIt is no coincidence that you are here tod...
14     \n\nThis motivational message encourages you ...
15    \n\nCommitting to your goals is key to making ...
16    \n\nIt is not necessary to exhaust yourself to...
17    \n\nEncouragement to appreciate your effor

* I see that my prompt to create a motivational message resulted in multiple texts with the word "motivation." In future iterations, I might edit the prompt.

* I also notice inconsistent newlines -- some text was returned with this whitespace, and some was not. I'm going to clean that up.

In [49]:
motivations = motivations.str.strip()

In [65]:
for motivation in motivations.head():
    print(motivation+'\n\n')

Motivate yourself to stay dedicated and consistent with your goals by showing up and not giving up. Effort is a vigorous and determined attempt to achieve your goals and can teach you that you can do more than you thought. With consistent effort, you will be able to improve yourself and reach success.


You did an amazing job today! Don't forget to grab your workout calendar and stretch calendar to stay on track. Put your goals into action and stay accountable to reach the results you want.


Motivational message: Taking the effort to prioritize yourself is not selfish, it is full of love for the self. You deserve to be the best in all areas of your life and it is worth putting in the effort to reach your goals. Everyone deserves to show up confident in all areas of their life and do not give up, the best version of yourself is waiting for you.


Motivation is a lifelong journey, and it is not a black and white conversation. Even if you take some time off, the important thing is to nev

* That is ... a lot. What happens if I whittle these down to one sentence each?

* I'll add a parameter for the number of sentences. (If I ever clean up this notebook for production code, there will be some serious refactoring.)

In [52]:
def get_response(text, sentences):
    '''
    Get response from ChatGPT with length that is the number of sentences
    '''
    directions = f'extract the motivational message from the following text in {sentences} sentences: '
    conditions = 'Ignore sentences that direct viewers to subscribe, like, buy, or comment.'
    response = openai.Completion.create(
        model='text-davinci-003',
        prompt=directions+text+conditions,
        temperature=0.7,
        max_tokens=100
    )
    return response

In [53]:
def get_motivation(text, sentences=3, sleep_time=3):
    '''
    Return the extracted motivation from the text in number of sentences, 
    pausing for sleep_time seconds after request.
    '''
    response = get_response(text, sentences)
    motivation = extract_text(response)
    time.sleep(sleep_time)
    return motivation

* Because I added a parameter, I have to use a lambda function to apply `get_motivation` to the series.

In [54]:
shorter_motivations = motivations.apply(lambda m: get_motivation(m, 1))
shorter_motivations[0]

'\n\nStay dedicated and consistent with your goals by putting in effort and not giving up, and you will be able to reach success.'

* I see the whitespace issue is showing up again. Will remove...

In [66]:
shorter_motivations = shorter_motivations.str.strip()
for s in shorter_motivations.head():
    print(s+'\n')

Stay dedicated and consistent with your goals by putting in effort and not giving up, and you will be able to reach success.

Stay focused on your goals and stay accountable to reach the results you want.

Prioritizing yourself and putting in the effort to reach your goals is worth it, so don't give up and strive to be the best version of yourself.

Keeping your motivation alive is key to making positive changes in life, so never give up and never quit on yourself even if you take some time off.

No matter how difficult the challenge, never give up and celebrate your successes along the way to reach your goals.



* This is better! I am reasonably satisfied with these summaries.

* One last thing I want to do is summarize the entire month of motivations into one, just to see what happens. Start by combining all of the responses into one, long text.

In [60]:
all_motivations = ' '.join(shorter_motivations)

In [61]:
all_motivations

"Stay dedicated and consistent with your goals by putting in effort and not giving up, and you will be able to reach success. Stay focused on your goals and stay accountable to reach the results you want. Prioritizing yourself and putting in the effort to reach your goals is worth it, so don't give up and strive to be the best version of yourself. Keeping your motivation alive is key to making positive changes in life, so never give up and never quit on yourself even if you take some time off. No matter how difficult the challenge, never give up and celebrate your successes along the way to reach your goals. Motivation is found in making the decision to keep going, even when feeling frustrated or tired, and recognizing that success is largely determined by persevering through failures. Motivational message: Prioritize your goals and dedicate your time to activities that will give you energy and benefit. Motivating yourself by reflecting on your reasons for working out and taking care o

* Now use that as the text to prompt ChatGPT3.

In [64]:
month_motivation = get_motivation(all_motivations, sentences=3)
month_motivation

' \n\nMotivational message: Prioritize your goals and dedicate your time to activities that will give you energy and benefit. Motivate yourself by reflecting on your reasons for working out and taking care of your body through proper warm-up, cool-down and recovery. Stay committed to your goals and put in the hard work and dedication, and you will be rewarded with success. Take the time to properly rest and recover, and use the strength and resilience you gain to celebrate your accomplishments'

[Return to top](#Extract-motivational-messages)

### Save the motivational messages

* Ok, there we have it. Even though ChatGPT did not *quite* follow directions with this four-sentence summary, its responses reflect the month's theme of Effort.

* Even though I don't know quite what to do with these summaries, I'm going to save them for future use.

In [70]:
summaries = data[['date','vid']]
summaries['summary'] = shorter_motivations
summaries.head()

,date,vid,summary
0,2023-01-02,qLKflJjWDi4,Stay dedicated and consistent with your goals ...
1,2023-01-03,NzIylrLhkJ4,Stay focused on your goals and stay accountabl...
2,2023-01-04,A5kHPgAi3I0,Prioritizing yourself and putting in the effor...
3,2023-01-06,A67VMX9hlmc,Keeping your motivation alive is key to making...
4,2023-01-07,lpv4RdNefyU,"No matter how difficult the challenge, never g..."


In [71]:
summaries.to_csv('../data/2023-01-effort-summaries.csv')

In [72]:
with open('../data/2023-01-effort-summary.txt', 'w') as file:
    file.write(month_motivation.strip())

[Return to top](#Extract-motivational-messages)

### Next steps

* I might transcribe the monthly summary into my workout journal.

* I might clean up the code to turn these notebooks into a script that I can run at the end of each month.

* I could go back to exploring statistics from Notebook 1 -- gather more data from the channel and create some visualizations.

* The YouTube channel metadata included urls for images. I could automate creation of a workout journal that includes pictures, workout descriptions, and motivational talks.

* It's also ok if there are no next steps! **I met my goal of getting my head around these new-to-me APIs and testing out the ChatGPT3 technology.**


[Return to top](#Extract-motivational-messages)